# Image Compression Using SVD

Image compression using Singular Value Decomposition (SVD) is a powerful technique from linear algebra that reduces the dimensionality of image data while retaining as much information as possible. Here’s a breakdown of how it works and how it’s implemented:

## **SVD of a Matrix**

For an image represented as a matrix \( A \) (a grayscale image with values between 0 and 255), the Singular Value Decomposition (SVD) is defined as:

\[
A = U \Sigma V^T
\]

- **\( U \) and \( V \):** Orthogonal matrices.  
- **\( \Sigma \):** Diagonal matrix of singular values, sorted in descending order.

---

### **Importance of Singular Values**
The singular values in \( \Sigma \) indicate the **amount of variance (or energy)** each contributes to the matrix.  
- **Larger singular values**: More significant in reconstructing the original image.

---

### **Compression via Truncation**

To compress the image:
1. Keep only the top \( k \) singular values (the most significant).
2. Approximate the original image \( A \) as:

\[
A_k = U_k \Sigma_k V_k^T
\]

Where:
- \( U_k \): The first \( k \) columns of



## **Key Components of the Code**

1. **Loading the Image**  
   - Convert the image to grayscale for simplicity.

2. **SVD Decomposition**  
   - Use `np.linalg.svd` to compute:  
     \[
     A = U \Sigma V^T
     \]

3. **Truncation**  
   - Keep only the top \( k \) singular values and their corresponding matrices \( U_k \), \( \Sigma_k \), and \( V_k^T \).

4. **Reconstruction**  
   - Approximate the original image using the truncated matrices:  
     \[
     A_k = U_k \Sigma_k V_k^T
     \]

---

## **How It Compresses Images**

### **1. Data Reduction**  
- Storing \( U_k \), \( \Sigma_k \), and \( V_k^T \) requires far fewer values compared to the original image matrix.

### **2. Quality vs. Compression Tradeoff**  
- **Low \( k \):** High compression, but lower image quality.  
- **Higher \( k \):** Less compression, but better visual reconstruction.

---

## **Additional Notes**
- This method works well for images with smooth regions and low detail.
- For **colored images**, the process is repeated for each color channel (**R, G, B**) independently.

---

### **Summary**
- Increasing \( k \) improves image quality but reduces compression efficiency.
- Balancing **quality** and **size** is key for effective image compression.


## Section 1: Black and white Images



In [1]:
# Importing the necessary libraries

from skimage import io
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [2]:
# Reading in the images in grayscale. Using the same set of images for both Black and White and Color image compression

bw_images = {
    "Bird" : io.imread('./images/Bird.jpg', as_gray=True),
    "Chandelier" : io.imread('./images/Chandelier.jpg', as_gray=True),
    "Flowers" : io.imread('./images/Flowers.jpg', as_gray=True),
    "Tree" : io.imread('./images/Tree.jpg', as_gray=True),
    "Waterfall" : io.imread('./images/Waterfall.jpg', as_gray=True)
}

In [3]:
# Showing image in widgets

def show_bw_images(Image_Name):
    fig = plt.figure(figsize=(18, 18))
    plt.title("Image Name: "+Image_Name+"\n")
    plt.imshow(bw_images[Image_Name], cmap = plt.cm.gray)  # picking the right color scale for display 
    plt.axis('off')
    plt.show()
    
interact(show_bw_images, Image_Name = list(bw_images.keys()))


interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.show_bw_images(Image_Name)>

In [4]:
# Checking the dimensions of each image to give an idea of how many singular values are present

for image in bw_images.keys():
    print(f"{image} - {bw_images[image].shape}")

Bird - (5184, 3456)
Chandelier - (3415, 5122)
Flowers - (7329, 5497)
Tree - (6000, 4000)
Waterfall - (3334, 5000)


In [ ]:
def compress_bw_image(Image_Name, k): 
    
    
    
   
    
    
    
    # Performing SVD and the compression
    U, S, V = np.linalg.svd(bw_images[Image_Name], full_matrices = False)
    u = U[:, 0:k]
    v = V[0:k]
    compressed_image = u @ np.diag(S[0:k]) @ v
    
    # fig = plt.figure(figsize=(18, 18))
    # fig.add_subplot(1, 2, 1)
    # plt.plot(S)
    # fig.add_subplot(1, 2, 2)
    # plt.plot(S)
    # plt.yscale('log')
    
    # anaysis 
    plt.figure(figsize=[16,6])
    plt.semilogy(S)
    plt.title("Singular values")
    plt.show()

    plt.figure(figsize=[16,6])
    plt.plot(np.cumsum(S)/np.sum(S))
    plt.plot(200, np.sum(np.diag(S[:200,:200]))/np.sum(S), marker='*', ls='none', ms=20, label='line with select markers')
    plt.title("Singular values cummulative sum")
    plt.show()
    
    
    fig = plt.figure(figsize=(18, 18))
    # Showing the original image
    fig.add_subplot(1, 2, 1)
    plt.title(f"Original Image - {Image_Name}")
    plt.imshow(bw_images[Image_Name], cmap = plt.cm.gray)
    plt.axis('off')
    
    # Showing the compressed image
    fig.add_subplot(1, 2, 2)
    plt.title(f"Compressed Image - {Image_Name} (Top {k} Singular values)")
    plt.imshow(compressed_image, cmap = plt.cm.gray)
    plt.axis('off')
    plt.show()
    
    # Showing the compression ratio
    dim1 = bw_images[Image_Name].shape
    size1 = dim1[0]*dim1[1]
    print(f"Original Image Size (in pixels)- {size1}")
    size2 = min(dim1)*k + k + k*max(dim1)
    print(f"Compressed Image Size (in pixels)- {size2}")
    ratio = round(size1/size2,2)
    print(f"Compression Ratio - {ratio}")    

    
    
interact(compress_bw_image, Image_Name = list(bw_images.keys()), k = (0,600))

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# Precompute the SVD of all images and store them in a dictionary (called when notebook is initialized)
svd_cache = {}

def compute_svd(Image_Name):
    """
    Computes the Singular Value Decomposition (SVD) of the image and caches the result.
    
    Parameters:
    Image_Name (str): The name of the image from the `bw_images` dictionary.
    """
    if Image_Name not in svd_cache:
        U, S, Vt = np.linalg.svd(bw_images[Image_Name], full_matrices=False)
        svd_cache[Image_Name] = (U, S, Vt)

def compress_bw_image(Image_Name, k): 
    """
    Compresses a black and white image using SVD by retaining the top k singular values
    and plots the analysis, original, and compressed images.
    
    Parameters:
    Image_Name (str): The name of the image from the `bw_images` dictionary.
    k (int): The number of singular values to retain for compression.
    """
    # Ensure that SVD has been computed for the image
    compute_svd(Image_Name)
    
    # Retrieve the precomputed SVD from cache
    U, S, Vt = svd_cache[Image_Name]
    
    # Retain the top k singular values and corresponding vectors
    U_k = U[:, :k]
    S_k = np.diag(S[:k])
    Vt_k = Vt[:k, :]
    
    # Reconstruct the compressed image
    compressed_image = U_k @ S_k @ Vt_k
    
    # Plot Singular values and Cumulative sum in two subplots
    fig, axs = plt.subplots(1, 2, figsize=[16,6])

    # Plot singular values on a semilog scale
    axs[0].semilogy(S)
    axs[0].set_title("Singular Values")
    
    # Plot cumulative sum of singular values
    axs[1].plot(np.cumsum(S) / np.sum(S))
    axs[1].plot(k, np.sum(S[:k]) / np.sum(S), marker='*', ls='none', ms=12, label=f'Top {k} Singular Values')
    axs[1].set_title("Cumulative Sum of Singular Values")
    axs[1].legend()

    plt.show()

    # Plot original and compressed images side by side
    fig = plt.figure(figsize=(18, 18))

    # Show the original image
    fig.add_subplot(1, 2, 1)
    plt.title(f"Original Image - {Image_Name}")
    plt.imshow(bw_images[Image_Name], cmap=plt.cm.gray)
    plt.axis('off')

    # Show the compressed image
    fig.add_subplot(1, 2, 2)
    plt.title(f"Compressed Image - {Image_Name} (Top {k} Singular Values)")
    plt.imshow(compressed_image, cmap=plt.cm.gray)
    plt.axis('off')
    plt.show()
    
    # Calculate and print the compression ratio
    dim1 = bw_images[Image_Name].shape
    size1 = dim1[0] * dim1[1]
    print(f"Original Image Size (in pixels): {size1}")
    
    size2 = dim1[0] * k + k + k * dim1[1]
    print(f"Compressed Image Size (in pixels): {size2}")
    
    ratio = round(size1 / size2, 2)
    print(f"Compression Ratio: {ratio}")

# Call compute_svd on all images once during initialization
for image_name in bw_images.keys():
    compute_svd(image_name)

# Example of calling the function interactively
interact(compress_bw_image, Image_Name=list(bw_images.keys()), k=(1, 600))


interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.compress_bw_image(Image_Name, k)>

## Section 2: Color Images

In [6]:
# This time, we'll be reading in the images normally. Using the same set of images for both Black and White and Color image compression

color_images = {
    "Bird" : io.imread('./images/Bird.jpg'),
    "Chandelier" : io.imread('./images/Chandelier.jpg'),
    "Flowers" : io.imread('./images/Flowers.jpg'),
    "Tree" : io.imread('./images/Tree.jpg'),
    "Waterfall" : io.imread('./images/Waterfall.jpg')
}

In [7]:
# Showing image in widgets

def show_color_images(Image_Name):
    fig = plt.figure(figsize=(18, 18))
    plt.title("Image Name: "+Image_Name+"\n")
    plt.imshow(color_images[Image_Name])  # picking the right color scale for display 
    plt.axis('off')
    plt.show()
    
interact(show_color_images, Image_Name = list(color_images.keys()))


interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.show_color_images(Image_Name)>

In [8]:

# Checking the dimensions of each image to give an idea of how many singular values are present

for image in color_images.keys():
    print(f"{image} - {color_images[image].shape}")

Bird - (5184, 3456, 3)
Chandelier - (3415, 5122, 3)
Flowers - (7329, 5497, 3)
Tree - (6000, 4000, 3)
Waterfall - (3334, 5000, 3)


Here you can see that there's a difference in the number of dimensions when compared to the Black and White images. Now you have 3 dimensions instead of 2, where the third dimension contains the color channel for the image (RGB). So, in order to perform compression, we need to do it individually for each color channel and then finally combine it.

In [9]:
def compress_color_image(Image_Name, k): 
        
    fig = plt.figure(figsize=(18, 18))
    
    # Showing the original image
    fig.add_subplot(1, 2, 1)
    plt.title(f"Original Image - {Image_Name}")
    plt.imshow(color_images[Image_Name])
    plt.axis('off')
    
    # Breaking the 3d matrix into 3 2d matrices - 1 for each color channel
    r = color_images[Image_Name][:,:,0]
    g = color_images[Image_Name][:,:,1]
    b = color_images[Image_Name][:,:,2]
    
    # Performing SVD for each matrix
    Ur, Sr, Vr = np.linalg.svd(r, full_matrices = True)
    Ug, Sg, Vg = np.linalg.svd(g, full_matrices = True)
    Ub, Sb, Vb = np.linalg.svd(b, full_matrices = True)
    
    # Compressing all the matrices individually
    ur = Ur[:, 0:k]
    vr = Vr[0:k]
    
    ug = Ug[:, 0:k]
    vg = Vg[0:k]
    
    ub = Ub[:, 0:k]
    vb = Vb[0:k]
    
    compressed_image_red = ur @ np.diag(Sr[0:k]) @ vr
    compressed_image_green = ug @ np.diag(Sg[0:k]) @ vg
    compressed_image_blue = ub @ np.diag(Sb[0:k]) @ vb
    
    # Combining the three 2D matrices into a single 3D matrix
    compressed_image = np.zeros(color_images[Image_Name].shape)
    compressed_image[:,:,0] = compressed_image_red
    compressed_image[:,:,1] = compressed_image_green
    compressed_image[:,:,2] = compressed_image_blue
    
    # Converting the values in the matrix to be in the range [0,255]
    compressed_image_final = np.array(compressed_image/np.amax(compressed_image)*255, np.int32)
  
    # Showing the compressed image
    fig.add_subplot(1, 2, 2)
    plt.title(f"Compressed Image - {Image_Name} (Top {k} Singular values)")
    plt.imshow(compressed_image_final.astype('uint8'))
    plt.axis('off')
    plt.show()
    
    # Showing the compression ratio
    dim1 = color_images[Image_Name].shape
    size1 = dim1[0]*dim1[1]
    print(f"Original Image Size (in pixels)- {size1}")
    size2 = min(dim1)*k + k + k*max(dim1)
    print(f"Compressed Image Size (in pixels)- {size2}")
    ratio = round(size1/size2,2)
    print(f"Compression Ratio - {ratio}")    

In [10]:
interact(compress_color_image, Image_Name = list(color_images.keys()), k = (0,600))

interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.compress_color_image(Image_Name, k)>

The difference between original and compressed images is more apparent for lower rank approximations for colored images than for Black and White images. 
But neverthless, using SVD, we can compress the image considerably by picking the top features of the image (top k singular values) without noticing visible changes in the quality of the image.

## Credits
https://web.stanford.edu/class/cs168/l/l9.pdf
http://timbaumann.info/svd-image-compression-demo/